In [3]:
import pandas as pd, numpy as np
from functions import *
LOCAL = True
if LOCAL:
    dataPath = '/home/zhenlan/Desktop/Projects/HMS/Data/'
dataFolder = 'train_eegs_LocalNorm'

In [2]:
df = pd.read_csv(dataPath+'train.csv')
TARGETS = df.columns[-6:].tolist()

In [3]:
# normalize prob
df[TARGETS] = df[TARGETS].values/df[TARGETS].sum(1).values[:,None]

In [4]:
out = df.groupby('eeg_id').agg({'eeg_label_offset_seconds': list,
                                    'seizure_vote': list,
                                    'lpd_vote': list,
                                    'gpd_vote': list,
                                    'lrda_vote': list,
                                    'grda_vote': list,
                                    'other_vote': list
                                }).reset_index()

In [ ]:
def process(x,mean=None,std=None):
    # fill NaN, standardize (global mean/std over all eegs)
    if mean is None:
        mean,std = np.nanmean(x,0),np.nanstd(x,0)
    x = (x-mean)/(std + 1e-8)
    return np.nan_to_num(x)

# Global Norm -> Consistency Across Patients
AllData = eegData(out,dataPath)
eegs = np.concatenate(AllData.eegs,0)
mean,std = np.nanmean(eegs,0),np.nanstd(eegs,0)
for id,array in zip(out.eeg_id.tolist(),AllData.eegs):
    np.save(dataPath+'train_eegs_globalNorm/'+str(id)+'.npy', process(array,mean,std))
    
# Local Norm -> EEG patterns can be influenced by individual anatomical and physiological differences. These can affect the EEG signal in ways that are not directly related to the neurological conditions being studied.
for id,array in zip(out.eeg_id.tolist(),AllData.eegs):
    np.save(dataPath+'train_eegs_LocalNorm/'+str(id)+'.npy', process(array))

In [5]:
train = out.iloc[:15000]
val = out.iloc[15000:]

In [10]:
train.to_pickle(dataPath+'train_data.pkl')
val.to_pickle(dataPath+'val_data.pkl')

In [13]:
# eegs,targets = next(iter(train_loader))
# eegs.shape,targets.shape
# (torch.Size([8, 10000, 20]), torch.Size([8, 6]))